In [15]:
# Uključivanje potrebnih biblioteka
import ipywidgets as widgets
from IPython.display import display
from IPython.core.getipython import get_ipython

# Lista dostupnih notebook-a
notebooks = [
    "Data_Analysis/10_most_common_criminal_offenses.ipynb",
    "crimes_over_year.ipynb",
    "trendiness.ipynb",
]

# Definisanje custom line magic komande za pokretanje notebook-a
def run_notebook(notebook_path):
    """Pokreće Jupyter notebook putem IPython magic komande `%run`."""
    get_ipython().run_line_magic('run', notebook_path)

# Kreiranje dropdown menija za izbor notebook-a
dropdown = widgets.Dropdown(options=notebooks, description='Izaberite notebook:')

# Kreiranje dugmeta za pokretanje izabranog notebook-a
button = widgets.Button(description="Pokreni")

# Funkcija koja se poziva kada korisnik klikne na dugme
def on_button_clicked(b):
    # Uzimanje putanje do izabranog notebook-a iz dropdown menija
    notebook_path = dropdown.value
    # Pokretanje izabranog notebook-a
    run_notebook(notebook_path)

# Dodavanje event handler-a na dugme
button.on_click(on_button_clicked)

# Prikazivanje dropdown menija i dugmeta
display(dropdown, button)


Dropdown(description='Izaberite notebook:', options=('Data_Analysis/10_most_common_criminal_offenses.ipynb', '…

Button(description='Pokreni', style=ButtonStyle())

FileNotFoundError: [Errno 2] No such file or directory: '../Resources/cleaned_crime_data.csv'

FileNotFoundError: [Errno 2] No such file or directory: '../Resources/cleaned_crime_data.csv'